## Feature Engineering

In [33]:
import pandas as pd
import numpy as np
import pydataset
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### 1. Load the tips dataset.



In [3]:
tips = pydataset.data('tips')
tips.head(1)

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2


#### a. Create a column named tip_percentage. This should be the tip amount divided by the total bill.

In [4]:
tips['tip_percentage'] = tips.tip / tips.total_bill
tips.head(1)

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447


#### b. Create a column named price_per_person. This should be the total bill divided by the party size.


In [19]:
tips = tips.rename(columns={'size': 'party_size'}) 
tips['price_per_person'] = tips.total_bill / tips.party_size
tips.head(5)

,total_bill,tip,sex,smoker,day,time,party_size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,6.147500


#### c. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?


* total_bill, day, time

#### d. Use select k best and recursive feature elimination to select the top 2 features for predicting tip amount. What are they?


In [23]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 1 to 244
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   total_bill        244 non-null    float64
 1   tip               244 non-null    float64
 2   sex               244 non-null    object 
 3   smoker            244 non-null    object 
 4   day               244 non-null    object 
 5   time              244 non-null    object 
 6   party_size        244 non-null    int64  
 7   tip_percentage    244 non-null    float64
 8   price_per_person  244 non-null    float64
dtypes: float64(4), int64(1), object(4)
memory usage: 27.2+ KB


In [26]:
tips.day.value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [38]:
X = tips.drop(columns=['tip', 'sex', 'smoker', 'day', 'time', 'tip_percentage'])
y = tips.tip

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=123)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [39]:
kbest = SelectKBest(f_regression, k=2)
kbest.fit(X_train_scaled, y_train)
kbest.fit(X_train_scaled, y_train)
X_train.columns[kbest.get_support()]

Index(['total_bill', 'party_size'], dtype='object')

#### e. Use select k best and recursive feature elimination to select the top 2 features for predicting tip percentage. What are they?


In [40]:
X_train_scaled

array([[ 2.22751067,  1.51285289,  0.67765502],
       [-0.4404687 , -0.57939047, -0.00598934],
       [-0.76989051, -0.57939047, -0.50426744],
       [-1.04984283, -0.57939047, -0.9277188 ],
       [-1.04197269, -0.57939047, -0.91581454],
       [ 0.02274559, -0.57939047,  0.6946611 ],
       [-0.70130781, -0.57939047, -0.40053036],
       [ 1.75979916,  1.51285289,  0.32392859],
       [ 1.71145398,  1.51285289,  0.28736552],
       [-1.07008035, -0.57939047, -0.95832974],
       [ 0.41850148, -0.57939047,  1.29327507],
       [-0.89918576, -0.57939047, -0.69983735],
       [ 0.66922183,  1.51285289, -0.50086623],
       [ 0.21275339,  1.51285289, -0.84608962],
       [ 1.65186573,  3.60509626, -0.72988142],
       [ 0.81538168, -0.57939047,  1.89358964],
       [ 0.83674351,  3.60509626, -1.14086165],
       [-0.99362751, -0.57939047, -0.84268841],
       [-0.77213912, -0.57939047, -0.50766866],
       [ 3.5080958 ,  0.46673121,  3.08628263],
       [-0.65183833, -0.57939047, -0.325

In [43]:
X_train.head()

,total_bill,party_size,price_per_person
24,39.42,4,9.855
191,15.69,2,7.845
210,12.76,2,6.380
11,10.27,2,5.135
197,10.34,2,5.170


In [42]:
# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, 2)

# fit the data using RFE
rfe.fit(X_train_scaled,y_train)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_features_to_select=2 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

#### f. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

### 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.


### 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.


### 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).